# 读取明细表数据

读取明细表，获取回款和合同两个表的数据

In [9]:
import pandas as pd
from pathlib import Path

# 假设你的路径已经定义好了
# PROJECT_ROOT = Path.cwd()
# DETAIL_TABLE_PATH = PROJECT_ROOT / "data" / "明细表.xlsx"

def read_detail_table():
    """读取明细表，返回两个 DataFrame 格式的表格"""
    if not DETAIL_TABLE_PATH.exists():
        print("文件不存在")
        return None, None
    
    # 使用 pandas 一次性读取所有工作表
    # sheet_name=None 表示读取所有 sheet，返回一个字典 {sheet名: DataFrame}
    all_sheets = pd.read_excel(DETAIL_TABLE_PATH, sheet_name=None)
    
    payment_df = pd.DataFrame() # 初始空表
    contract_df = pd.DataFrame()
    
    # 获取所有 sheet 的名称列表
    sheet_names = list(all_sheets.keys())

    for i, name in enumerate(sheet_names):
        df = all_sheets[name]
        
        # 逻辑判断：根据索引（第1个是回款，第2个是合同）或名称
        if i == 0 or "回款" in name:
            payment_df = df
        elif i == 1 or "合同" in name:
            contract_df = df
            
    return payment_df, contract_df

# --- 调用并查看结果 ---
pay_table, con_table = read_detail_table()


In [16]:
con_table.to_csv('con_table.csv', index=False)
pay_table.to_csv('pay_table.csv', index=False)


In [15]:
pay_table.fillna(0)

,负责人,公司名称,项目类型,项目名称,月初预计可能回款,月初预计确定回款,可能回款,确定回款,实际回款,回款节点确定,回款节点,未回款金额,未完成原因,解决办法
0,李四,B公司,软件开发,项目1,50000,49000,41000,36000,34000,验收结算流程,验收结算流程,2000,客户流程慢,催款或调整节点
1,李六,C公司,技术服务,项目2,29000,5000,47000,40000,16000,验收结算流程,验收结算流程,24000,客户流程慢,催款或调整节点
2,王五,C公司,技术服务,项目3,41000,10000,19000,10000,43000,验收结算流程,验收结算流程,0,0,0
3,李四,B公司,技术服务,项目4,8000,29000,37000,39000,33000,0,0,6000,客户流程慢,催款或调整节点
4,李四,B公司,软件开发,项目5,18000,29000,19000,50000,37000,0,0,13000,客户流程慢,催款或调整节点
5,李四,B公司,软件开发,项目6,6000,37000,25000,15000,17000,挂账流程,挂账流程,0,0,0
6,李四,B公司,设备采购,项目7,0,0,49000,49000,49000,付款计划发起,付款计划发起,0,0,0
7,张三,A公司,技术服务,项目8,16000,7000,45000,34000,45000,挂账流程,挂账流程,0,0,0
8,王五,C公司,设备采购,项目9,35000,12000,29000,28000,3000,付款计划发起,付款计划发起,25000,客户流程慢,催款或调整节点
9,赵起,B公司,软件开发,项目10,39000,5000,29000,45000,37000,0,0,8000,客户流程慢,催款或调整节点


## 第一个汇总表

In [2]:
def generate_summary(df, numeric_cols):
    """
    通用汇总函数：按公司名称和项目类型汇总，并添加公司小计
    :param df: 原始表 (pay_table 或 con_table)
    :param numeric_cols: 需要累加的数值列清单
    """
    # 1. 只提取必要的列，自动过滤掉负责人、项目名称等
    cols_to_keep = ['公司名称', '项目类型'] + numeric_cols
    # 过滤掉不在原表中的列，防止报错
    df_filtered = df[[c for c in cols_to_keep if c in df.columns]].copy()
    
    # 2. 按公司名称和项目类型进行初步汇总
    summary = df_filtered.groupby(['公司名称', '项目类型'])[numeric_cols].sum(numeric_only=True).reset_index()
    
    final_list = []
    # 3. 按照公司名称分组，循环插入小计行
    for company, group in summary.groupby('公司名称'):
        # 添加该公司的明细汇总数据
        final_list.append(group)
        
        # 计算该公司的小计行
        subtotal = group[numeric_cols].sum().to_frame().T
        subtotal['公司名称'] = f"{company} 小计"
        subtotal['项目类型'] = "---" # 小计行项目类型显示为横杠
        
        final_list.append(subtotal)
    
    # 4. 合并所有行
    return pd.concat(final_list, ignore_index=True)

# ================= 使用方法 =================

# 1. 处理回款表
pay_cols = ['月初预计可能回款', '月初预计确定回款', '可能回款', '确定回款', '实际回款']
pay_summary_result = generate_summary(pay_table, pay_cols)

# 2. 处理合同表
con_cols = ['月初预计可能合同', '月初预计确定合同', '可能合同', '确定合同', '实际合同']
con_summary_result = generate_summary(con_table, con_cols)
final_columns = ['公司名称', '项目类型'] + pay_cols + con_cols
combined_result = pd.merge(
    pay_summary_result, 
    con_summary_result, 
    on=['公司名称', '项目类型'], 
    how='outer'
)

# 2. 清理数据
# 合并后，如果某一边没有数据会显示为 NaN，建议填充为 0 方便阅读
combined_result = combined_result.fillna(0)
combined_result

## 第二个

In [ ]:
import pandas as pd

# --- 前置数据处理保持不变 ---
def process_table(df, numeric_cols):
    keys = ['公司名称', '负责人', '项目类型']
    available_cols = [c for c in (keys + numeric_cols) if c in df.columns]
    df_filtered = df[available_cols].copy()
    return df_filtered.groupby(keys)[numeric_cols].sum(numeric_only=True).reset_index()

# 1. 定义列名
pay_cols = ['月初预计可能回款', '月初预计确定回款', '可能回款', '确定回款', '实际回款']
con_cols = ['月初预计可能合同', '月初预计确定合同', '可能合同', '确定合同', '实际合同']
all_numeric_cols = pay_cols + con_cols

# 2. 获取初步汇总数据 (公司-负责人-项目)
pay_summary = process_table(pay_table, pay_cols)
con_summary = process_table(con_table, con_cols)

# 3. 拼接两表
combined_result = pd.merge(
    pay_summary, con_summary, 
    on=['公司名称', '负责人', '项目类型'], 
    how='outer'
).fillna(0)

# 4. 生成最终的三层汇总结构

final_list = []

# 按公司名称分组
for company, group in combined_result.groupby('公司名称', sort=False):
    # --- A. 负责人明细层 ---
    # 确保排序是 [负责人 -> 项目类型]
    details = group.sort_values(['负责人', '项目类型']).copy()
    final_list.append(details)
    
    # --- B. 公司各项目汇总层 (对应图片紫色部分) ---
    type_summary = group.groupby('项目类型')[all_numeric_cols].sum().reset_index()
    type_summary['公司名称'] = company
    type_summary['负责人'] = f"{company}汇总"
    final_list.append(type_summary)
    
    # --- C. 公司最终小计 (对应图片黄色部分) ---
    subtotal = group[all_numeric_cols].sum().to_frame().T
    subtotal['公司名称'] = company
    subtotal['负责人'] = "小计"
    subtotal['项目类型'] = "---"
    final_list.append(subtotal)

# 2. 合并所有层级
full_df = pd.concat(final_list, ignore_index=True)

# 3. 【关键步骤】转换为多级索引实现“视觉合并”
# 将需要合并的列设为索引。Pandas 在展示时会自动合并重复的索引项。
display_df = full_df.set_index(['公司名称', '负责人', '项目类型'])

display_df

## 分类

In [5]:
def split_by_company(df):
    """
    将输入的 DataFrame 按照 '公司名称' 拆分为多个表
    返回一个字典：{公司名: 对应的完整数据表}
    """
    if df is None or df.empty:
        return {}
    
    # 获取当前表里所有的公司
    companies = df['公司名称'].unique()
    
    # 拆分：每个公司一个独立的 DataFrame，保留所有列
    company_tables = {name: df[df['公司名称'] == name].copy() for name in companies}
    
    return company_tables

# 2. 执行拆分
# 拆分后的结果是两个字典
pay_tables_dict = split_by_company(pay_table)
con_tables_dict = split_by_company(con_table)

NameError: name 'payment_data' is not defined

In [4]:
import pandas as pd
from pathlib import Path
from openpyxl.styles import Alignment, Font, PatternFill
from openpyxl.utils import get_column_letter

# ================= 配置区域 =================
OUTPUT_FILE = Path.cwd() / "data" / "全公司汇总报表_类别汇总版.xlsx"

COL_CONFIG = {
    'pay': ['月初预计可能回款', '月初预计确定回款', '可能回款', '确定回款', '实际回款'],
    'con': ['月初预计可能合同', '月初预计确定合同', '可能合同', '确定合同', '实际合同']
}

# 颜色配置
TYPE_SUMMARY_FILL = PatternFill("solid", fgColor="E2EFDA") # 淡绿色用于项目类别汇总
GRAND_TOTAL_FILL = PatternFill("solid", fgColor="FFF2CC")  # 淡黄色用于公司总计

# ================= 逻辑处理函数 =================

def calculate_final_summaries(df, numeric_cols):
    """
    逻辑：明细行展示 -> 底部按项目类型汇总 -> 底部公司总计
    """
    if df is None or df.empty:
        return pd.DataFrame()
    
    df = df.copy()
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # 1. 整理明细行 (按负责人、项目类型排序)
    details = df.sort_values(['负责人', '项目类型', '项目名称']).copy()
    details['_row_type'] = 'DETAIL'
    
    # 2. 计算【项目类别汇总】 (在所有明细最后展示)
    type_summary = df.groupby('项目类型')[numeric_cols].sum().reset_index()
    type_summary['公司名称'] = df['公司名称'].iloc[0]
    type_summary['负责人'] = '项目类别汇总' # 负责人列显示此标签
    type_summary['项目名称'] = '汇总'
    type_summary['_row_type'] = 'TYPE_SUMMARY'
    
    # 3. 计算【公司总计】
    grand_total = df[numeric_cols].sum().to_frame().T
    grand_total['公司名称'] = df['公司名称'].iloc[0]
    grand_total['负责人'] = '公司总计'
    grand_total['项目类型'] = '---'
    grand_total['项目名称'] = '---'
    grand_total['_row_type'] = 'GRAND_TOTAL'
    
    # 按照顺序拼接：明细 -> 类别汇总 -> 公司总计
    return pd.concat([details, type_summary, grand_total], ignore_index=True)

def apply_excel_formatting(ws, df, title, start_row):
    """
    格式化：合并标题、合并负责人、汇总行上色
    """
    if df.empty: return start_row
    
    max_col_idx = len(df.columns) - 1 # 排除 _row_type 列
    data_header_row = start_row + 2
    data_start_row = data_header_row + 1
    
    # 1. 大标题处理
    ws.merge_cells(start_row=start_row + 1, start_column=1, end_row=start_row + 1, end_column=max_col_idx)
    cell = ws.cell(row=start_row + 1, column=1, value=title)
    cell.alignment = Alignment(horizontal='center', vertical='center')
    cell.font = Font(bold=True, size=14)
    
    # 2. 处理样式与负责人合并
    m_start_person = data_start_row
    
    for i in range(len(df)):
        excel_row = data_start_row + i
        row_data = df.iloc[i]
        row_type = row_data.get('_row_type')
        
        # A. 上色逻辑
        if row_type == 'TYPE_SUMMARY':
            for c in range(1, max_col_idx + 1):
                ws.cell(excel_row, c).fill = TYPE_SUMMARY_FILL
        elif row_type == 'GRAND_TOTAL':
            for c in range(1, max_col_idx + 1):
                ws.cell(excel_row, c).fill = GRAND_TOTAL_FILL
        
        # B. 负责人合并逻辑 (第一列)
        curr_person = ws.cell(excel_row, 1).value
        next_person = ws.cell(excel_row + 1, 1).value if i < len(df) - 1 else None
        
        # 只有在明细行时才合并负责人姓名
        # 如果当前或下一行是汇总/总计，则停止合并
        if curr_person != next_person:
            if excel_row > m_start_person:
                ws.merge_cells(start_row=m_start_person, start_column=1, end_row=excel_row, end_column=1)
                ws.cell(m_start_person, 1).alignment = Alignment(horizontal='center', vertical='center')
            m_start_person = excel_row + 1

    # 3. 隐藏最后一列标记列
    ws.column_dimensions[get_column_letter(len(df.columns))].hidden = True
    return start_row + len(df) + 5

# ================= 主流程 =================

def generate_category_summary_report(pay_table, con_table):
    pay_dict = {n: pay_table[pay_table['公司名称'] == n].copy() for n in pay_table['公司名称'].unique()} if pay_table is not None else {}
    con_dict = {n: con_table[con_table['公司名称'] == n].copy() for n in con_table['公司名称'].unique()} if con_table is not None else {}
    
    all_companies = sorted(list(set(pay_dict.keys()) | set(con_dict.keys())))
    
    with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
        for company in all_companies:
            sheet_name = str(company).replace("/", "_")[:31]
            curr_row = 0
            
            # 处理回款块
            df_p = calculate_final_summaries(pay_dict.get(company), COL_CONFIG['pay'])
            if not df_p.empty:
                df_p.to_excel(writer, sheet_name=sheet_name, index=False, startrow=curr_row + 1)
                curr_row = apply_excel_formatting(writer.book[sheet_name], df_p, f"【{company} - 回款明细及类别汇总】", curr_row)
            
            # 处理合同块
            df_c = calculate_final_summaries(con_dict.get(company), COL_CONFIG['con'])
            if not df_c.empty:
                df_c.to_excel(writer, sheet_name=sheet_name, index=False, startrow=curr_row + 1)
                apply_excel_formatting(writer.book[sheet_name], df_c, f"【{company} - 合同明细及类别汇总】", curr_row)

    print(f"✅ 报表已生成！\n路径: {OUTPUT_FILE}")

# 执行
generate_category_summary_report(pay_table, con_table)

NameError: name 'contract_data' is not defined